In [36]:
import numpy as np
import riskslim
import cplex as cplex

import pandas as pd

from sklearn.metrics import accuracy_score, roc_auc_score

from pipeline import my_pipeline

In [37]:
def transform_dataset(pd_data):
    """
    Takes in a pandas dataframe, sets the constraints and settings for RiskSLIM 
    and returns a transformed dataset (which can be used by RiskSLIM).
    """
    # load dataset
    data = riskslim.load_data_from_csv(pandas_dataset=pd_data)
    N, P = data['X'].shape

    # problem parameters
    max_coefficient = 5                                         # value of largest/smallest coefficient
    max_L0_value = 5                                            # maximum model size
    max_offset = 50                                             # maximum value of offset parameter (optional)
    c0_value = 1e-6                                             # L0-penalty parameter such that c0_value > 0; larger values -> sparser models; we set to a small value (1e-6) so that we get a model with max_L0_value terms
    w_pos = 1.00                                                # relative weight on examples with y = +1; w_neg = 1.00 (optional)

    # coefficient set
    coef_set = riskslim.CoefficientSet(variable_names = data['variable_names'], lb=-max_coefficient, ub=max_coefficient, sign=0)
    coef_set.update_intercept_bounds(X = data['X'], y = data['Y'], max_offset = max_offset)

    # create constraint dictionary
    N, P = data['X'].shape
    trivial_L0_max = P - np.sum(coef_set.C_0j == 0)
    max_L0_value = min(max_L0_value, trivial_L0_max)

    constraints = {
        'L0_min': 0,
        'L0_max': max_L0_value,
        'coef_set': coef_set,
    }

    # Run RiskSLIM
    settings = {
        #
        'c0_value': c0_value,
        'w_pos': w_pos,
        #
        # LCPA Settings
        'max_runtime': 300.0,                               # max runtime for LCPA
        'max_tolerance': np.finfo('float').eps,             # tolerance to stop LCPA (set to 0 to return provably optimal solution)
        'display_cplex_progress': True,                     # set to True to print CPLEX progress
        'loss_computation': 'lookup',                       # how to compute the loss function ('normal','fast','lookup')
        #
        # Other LCPA Heuristics
        'chained_updates_flag': True,                       # use chained updates
        'add_cuts_at_heuristic_solutions': True,            # add cuts at integer feasible solutions found using polishing/rounding
        #
        # LCPA Rounding Heuristic
        'round_flag': False,                                # round continuous solutions with SeqRd
        'polish_rounded_solutions': True,                   # polish solutions rounded with SeqRd using DCD
        'rounding_tolerance': float('inf'),                 # only solutions with objective value < (1 + tol) are rounded
        'rounding_start_cuts': 0,                           # cuts needed to start using rounding heuristic
        'rounding_start_gap': float('inf'),                 # optimality gap needed to start using rounding heuristic
        'rounding_stop_cuts': 20000,                        # cuts needed to stop using rounding heuristic
        'rounding_stop_gap': 0.2,                           # optimality gap needed to stop using rounding heuristic
        #
        # LCPA Polishing Heuristic
        'polish_flag': False,                                # polish integer feasible solutions with DCD
        'polishing_tolerance': 0.1,                         # only solutions with objective value (1 + tol) are polished.
        'polishing_max_runtime': 10.0,                      # max time to run polishing each time
        'polishing_max_solutions': 5.0,                     # max # of solutions to polish each time
        'polishing_start_cuts': 0,                          # cuts needed to start using polishing heuristic
        'polishing_start_gap': float('inf'),                # min optimality gap needed to start using polishing heuristic
        'polishing_stop_cuts': float('inf'),                # cuts needed to stop using polishing heuristic
        'polishing_stop_gap': 0.0,                          # max optimality gap required to stop using polishing heuristic
        #
        # Initialization Procedure
        'initialization_flag': True,                        # use initialization procedure
        'init_display_progress': True,                      # show progress of initialization procedure
        'init_display_cplex_progress': False,               # show progress of CPLEX during intialization procedure
        #
        'init_max_runtime': 300.0,                          # max time to run CPA in initialization procedure
        'init_max_iterations': 10000,                       # max # of cuts needed to stop CPA
        'init_max_tolerance': 0.0001,                       # tolerance of solution to stop CPA
        'init_max_runtime_per_iteration': 300.0,            # max time per iteration of CPA
        'init_max_cplex_time_per_iteration': 10.0,          # max time per iteration to solve surrogate problem in CPA
        #
        'init_use_rounding': True,                          # use Rd in initialization procedure
        'init_rounding_max_runtime': 30.0,                  # max runtime for Rd in initialization procedure
        'init_rounding_max_solutions': 5,                   # max solutions to round using Rd
        #
        'init_use_sequential_rounding': True,               # use SeqRd in initialization procedure
        'init_sequential_rounding_max_runtime': 10.0,       # max runtime for SeqRd in initialization procedure
        'init_sequential_rounding_max_solutions': 5,        # max solutions to round using SeqRd
        #
        'init_polishing_after': True,                       # polish after rounding
        'init_polishing_max_runtime': 30.0,                 # max runtime for polishing
        'init_polishing_max_solutions': 5,                  # max solutions to polish
        #
        # CPLEX Solver Parameters
        'cplex_randomseed': 0,                              # random seed
        'cplex_mipemphasis': 0,                             # cplex MIP strategy
    }

    return data, constraints, settings


In [38]:
# Import data
data = "data/compas-scores-two-years-features.csv"
data = pd.read_csv(data)
data = my_pipeline(data, 40)

data, constraints, settings = transform_dataset(data)
model_info, _, _ = riskslim.run_lattice_cpa(data, constraints, settings)
riskslim.print_model(model_info['solution'], data)

c:\Users\zanme\miniconda3\envs\riskslim\lib\site-packages\Orange\data\pandas_compat.py:193: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(s, utc=True)
c:\Users\zanme\miniconda3\envs\riskslim\lib\site-packages\Orange\data\pandas_compat.py:193: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(s, utc=True)
c:\Users\zanme\miniconda3\envs\riskslim\lib\site-packages\Orange\data\pandas_compat.py:193: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(s, utc=True)
c:\Users\zanme\miniconda3\envs\riskslim\lib\site-packages\Orange\data\pandas_compa

setting c0_value = 0.0 for (Intercept) to ensure that intercept is not penalized
05/19/24 @ 01:21 PM | 251 rows in lookup table
05/19/24 @ 01:21 PM | ------------------------------------------------------------
05/19/24 @ 01:21 PM | runnning initialization procedure
05/19/24 @ 01:21 PM | ------------------------------------------------------------
05/19/24 @ 01:21 PM | CPA produced 2 cuts
05/19/24 @ 01:21 PM | running naive rounding on 3 solutions
05/19/24 @ 01:21 PM | best objective value: 0.6931
05/19/24 @ 01:21 PM | rounding produced 3 integer solutions
05/19/24 @ 01:21 PM | best objective value is 0.6931
05/19/24 @ 01:21 PM | running sequential rounding on 3 solutions
05/19/24 @ 01:21 PM | best objective value: 0.6931
05/19/24 @ 01:21 PM | sequential rounding produced 1 integer solutions
05/19/24 @ 01:21 PM | best objective value: 0.6931
05/19/24 @ 01:21 PM | polishing 4 solutions
05/19/24 @ 01:21 PM | best objective value: 0.6931
05/19/24 @ 01:21 PM | polishing produced 2 integer 

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
05/19/24 @ 01:21 PM | adding 238 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6931.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

Pr(Y = +1) = 1.0/(1.0 + exp(-(5 + score)),,
===========================================,=================,=========
age=< 20,5 points,+ .....
age=20 - 40,1 points,+ .....
priors_count=20 - 30,1 points,+ .....
priors_count=< 10,-1 points,+ .....
juv_fel_count=< 5,-5 points,+ .....
===========================================,=================,=========
ADD POINTS FROM ROWS 1 to 5,SCORE,= .....


In [39]:
# Extract model coefficients
coefficients = model_info['solution']
feature_coefficients = coefficients

# Get the features and labels from the dataset
X = data['X']  # assuming 'X' is the key for features
y_true = data['Y']  # assuming 'Y' is the key for the target variable

# Calculate the score for each instance in the validation set
scores = np.sum(data["X"] * model_info["solution"], axis=1)

# Calculate the probability for each instance
probabilities = 1.0 / (1.0 + np.exp(-scores))

# Calculate the predicted class for each instance
predictions = (probabilities > 0.5).astype(int)

# Change all the -1 values to 0 in Y for validation data
y = data["Y"].copy()
y[y == -1] = 0

# Calculate the AUC, accuracy, sensitivity, and specificity
auc = roc_auc_score(y, scores)
accuracy = accuracy_score(y, predictions)

print(f"AUC: {auc}")
print(f"Accuracy: {accuracy}")

AUC: 0.6239439670846438
Accuracy: 0.5948156362628223
